# Instructional Airplane Leasing Program

## Overview

This dataset details every accident investigated by the National Transportation Safety Board over the last 40 years.  This data details the make, models, weather conditions, flight phases, injuries, and deaths during each accident.  The data could potentially help organizations find correlations, and identify conditions of an airplane flight that make it safer.  

In this analyis, we will use the data to help a flight school choose safe airplances for instructional flights.  Overall, we have these three findings:
1. **Cessna** and **Piper** airplanes appear to offer better protection to its passengers during airplane accidents.
2. Accidents that occur during **taxi**, **takeoff, and landing have the lowest  fatalities.  Accidents that occur during maneuvers and cruise phases have the highest fatality rates. 
3. Flights occurring during the summer months tend to have lower fatality rates that flights occuring during the winter months.

## Business Problem

In order to choose safe airplanes for the flight school, we will investigate these business questions:
1. How does the make of the aircraft affect airplane safety?
2. How does the phase of flight affect the survivability of airplane accidents?
3. How do the time of year and weather conditions affect the survival rate an airplane accident? 

To measure airplane safety, we will consider the protection offered by the airplane during an accident.  We will look at this three different ways
- **Survive:** The fraction of accidents with no fatalities.  
- **Survival Rate:** The fraction of passengers killed during an accident
- **Injury Rate:** The fraction of passengers uninjured during an accident. 

## Data Understanding

## Intake Data ?

In this analysis we read in the entire AviationData dataset.  For each record, we use the following data:
- Event Date
- Make of Airplane
- Accidents injured, killed, minor injuries, and uninjured passengers in each accident
- Phase of Flight and weather conditions

## Outcome Data

## Analysis

### Airplane Safety by Make of Aircraft

### Airplane Safety during Phases of Flight

### Airplane Safety during Different Months of the Year

## Conclusions

## Next Steps